In [ ]:
# Loading the IMDB dataset
from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

# Encoding the integer sequence into a binary matrix
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

# Define the model
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# Prepare a validation set
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

# Train the model
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

history = model.fit(partial_x_train, partial_y_train, epochs=20, batch_size=512, validation_data=(x_val, y_val))

# Plotting the training and validation loss
import matplotlib.pyplot as plt

history_dict = history.history
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(loss)+1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

# Plotting the training and validation accuracy
plt.clf()
acc = history_dict['acc']
val_acc = history_dict['val_acc']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

### 영화 리뷰 분류: 이진 분류 예제
* IMDB 데이터셋 사용
    * 5만개의 리뷰 데이터, 훈련 데이터 2만 5,000개와 테스트 데이터 2만 5,000개로 나뉘어 있고 각각 50% 부정 리뷰와 50% 긍정 리뷰로 구성
* `num_word=10000` 매개변수는 훈련 데이터에서 가장 자주 나타나는 단어 1만 개만 사용하겠다는 의미
    * 드물게 나타나는 단어는 무시
    * 적절한 크기의 벡터 데이터를 얻을 수 있음
* 변수 `train_data`와 `test_data`는 리뷰 목록, 각 리뷰는 단어 인덱스의 리스트
* `train_labels`와 `test_labels`는 **부정**을 나타내는 0과 **긍정**을 나타내는 1의 리스트
* 리스트를 텐서로 바꾸는 두가지 방법 (신경망에 숫자 리스트를 주입할 수 없음)
    * 같은 길이가 되도록 리스트에 패딩(padding)을 추가하고 (sample, sequence_length) 크기의 정수 텐서로 변환, 그다음 이 정수 텐서를 다룰 수 있는 층을 신경망의 첫번째 층으로 사용 (Embedding 층)
    * 리스트를 원-핫 인코딩하여 0과 1의 벡터로 변환, 예를 들어 시퀀스 [3, 5]를 인덱스 3과 5의 위치를 1이고 그 외는 모두 0인 10,000차원의 벡터로 각각 변환 후 부동 소수 벡터 데이터를 다룰 수 있는 Dense 층을 신경망의 첫 번째 층으로 사용
* 입력 데이터가 벡터고 레이블은 스칼라(1 또는 0), 이런 문제에서 잘 작동하는 네트워크 종류는 `relu` 활성화 함수를 사용한 완전 연결 층(`Dense(16, activation='relu')`)을 그냥 쌓은 것
* 전달된 매개변수(16)는 **은닉 유닛(hidden unit)**의 개수
    * 하나의 은닉 유닛은 층이 나타내는 표현 공간에서 하나의 차원
    * 16개의 은닉 유닛이 있다는 것은 가중치 행렬 W의 크기가 (input_dimension, 16)이라는 뜻
    * 입력 데이터와 W를 점곱하면 입력 데이터가 16차원으로 표현된 공간으로 투영됨 (그리고 편향 벡터 b를 더하고 relu 연산을 적용)
    * 은닉 유닛을 늘리면(표현 차원 공간을 더 고차원으로 만들면) 신경망이 더욱 복잡한 표현을 학습할 수 있지만 계산 비용이 커지고 원하지 않는 패턴을 학습할 수도 있음(훈련 데이터에서는 성능이 향상되지만 테스트 데이터에서는 그렇지 않는 패턴)
* `Dense` 층을 쌓을 때 두 가지 중요한 구조상의 결정이 필요
    * 얼마나 많은 층을 사용할 것인가?
    * 각 층에 얼마나 많은 은닉 유닛을 둘 것인가?
    * 이번 문제에서는 16개의 은닉 유닛을 가진 2개의 은닉 층
    * 현재 리뷰의 감정을 스칼라 값의 예측으로 출력하는 세 번째 층
* 중간에 있는 은닉 층은 활성화 함수로 relu를 사용하고 마지막 층은 확률(0과 1 사이의 점수고, 어떤 샘플이 타깃 '1'일 가능성이 높다는 것은 그 리뷰가 긍정일 가능성이 높다는 것을 의미)을 출력하기 위해 시그모이드 활성화 함수를 사용
    * `relu`는 음수를 0으로 만드는 함수
    * `sigmoid`는 임의의 값을 [0, 1] 사이로 압축하므로 출력 값을 확률처럼 해석할 수 있음
* 마지막으로 손실 함수와 옵티마이저 선택
* 이진 불류 문제고 신경망의 출력이 확률이기 때문에 `binary_crossentropy` 손실이 적합
    * `mean_squared_error`도 사용 가능
    * 확률을 출력하는 모델을 사용할 때는 크로스엔트로피가 최선
        * **크로스엔트로피**는 정보 이론 분야에서 온 개념으로 확률 분포 간의 차이를 측정
        * 여기에서는 원본 분포와 예측 분포 사이를 측정
* `rmsprop` 옵티마이저 사용
* 훈련하는 동안 처음 본 데이터에 대한 모델의 정확도를 측정하기 위해서는 원본 훈련 데이터에서 10,000개의 샘플을 떼어 검증 세트를 만들어야 함
* 모델을 512개의 샘플씩 미니 배치를 만들어 20번의 에포크 동안 훈련시킨다, 동시에 따로 떼어 놓은 1만 개의 샘플에서 손실과 정확도를 측정
* 결과에서 보듯이 훈련 손실이 에포크마다 감소하고 훈련 정확도는 에포크마다 증가
    * 경사 하강법 최적화를 사용했을 때 반복마다 최소화되는 것이 손실이므로 기대했던 대로임
* 검증 손실과 정확도는 네 번째 에포크에서 그래프가 역전되는 것으로 보임
    * **과대적합(overfitting)**
    * 두 번째 에포크 이후부터 훈련 데이터에 과도하게 최적화되어 훈련 데이터에 특화된 표현을 학습하므로 훈련 세트 이외의 데이터에는 일반화되지 못함
    * 과대적합을 방지하기 위해서 세 번째 에포크 이후에 훈련을 중지할 수 있음